In [7]:
import numpy as np
import torch
print(torch.__version__)
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
torch.cuda.is_available()
from pathlib import Path
import time
import datasets
import torch.nn.functional as F

model_training='tokemonster'
LIMIT_DATASET = 100_000
RANDOM_SEED = 42
NUM_TOKENIZER_TRAINING_ITEMS = 1_000_000  # I made this up, but it seems reasonable
if model_training=='bert':
    VOCAB_SIZE = 32_768  # from Cramming
    DEVICE_BATCH_SIZE = 100 # aprox 128, adjust to get near 100% gpu memory use
    MODEL_MAX_SEQ_LEN = 128  # from Cramming
else:
    VOCAB_SIZE = 16_000  # tokenmonster
    # VOCAB_SIZE = 1_024  # tokenmonster
    DEVICE_BATCH_SIZE = 1 # Token monster aguenta um batch size de (200-248)!! Geralmente melhora a qualidade do treino
    MODEL_MAX_SEQ_LEN = 84  # token_monster
    RETRIEVAL_NUM=5
    # DEVICE_BATCH_SIZE=12
    # MODEL_MAX_SEQ_LEN = 1000

MASK_ID=4
PAD_ID=0
gradient_accumulation_steps = 2048 // DEVICE_BATCH_SIZE  # roughly based on Cramming
batch_size = DEVICE_BATCH_SIZE * gradient_accumulation_steps
print('batch_size: ',batch_size)
RUN_DIR = Path("data") / f"run_{time.strftime('%Y%m%d-%H%M%S')}"
CHECKPOINT_DIR = RUN_DIR / "training_checkpoints"
MODEL_DIR = RUN_DIR / "model"
TOKENIZER_PATH = RUN_DIR / "tokenizer.json"
TRAINER_HISTORY_PATH = RUN_DIR / "trainer_history.json"
RUN_DIR.mkdir(exist_ok=True, parents=True)

dataset = datasets.load_dataset(
    "sradc/chunked-shuffled-wikipedia20220301en-bookcorpusopen",
    split=f"train[:{LIMIT_DATASET}]" if LIMIT_DATASET else "train",
    revision="0e6fada2dd43136e4a3f637da41de2e596aee674",
)
print('loaded dataset!!')



2.0.1+cu117
batch_size:  2048


Resolving data files: 100%|██████████| 53/53 [00:00<00:00, 69.03it/s]


loaded dataset!!


In [3]:
import data_clusterize
texts=data_clusterize.get_texts(dataset)
tfidf_model, vecs = data_clusterize.train_vectorizer(texts)

  0%|          | 0/10000 [00:00<?, ?it/s]/home/miu/miniconda3/envs/torch/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['number'] not in stop_words.
  warnings.warn(
100%|██████████| 10000/10000 [00:00<00:00, 29866.09it/s]


In [4]:
cluster_labels=data_clusterize.clusterize(vecs,N_DOMAINS=2)

500it [00:01, 451.17it/s]
500it [00:00, 19580.34it/s]


In [5]:
len(cluster_labels)

10000

In [6]:
corpus=[]
for i in range(len(cluster_labels)):
    if cluster_labels[i]==0:
        corpus.append(texts[i])
len(corpus)

4463

In [8]:
import data_clusterize

corpus=data_clusterize.get_texts(dataset)

In [ ]:
query=''

In [9]:
import string
from fastbm25 import fastbm25
corpus = [doc.translate(str.maketrans('', '', string.punctuation)).replace('\n',"").lower().split() for doc in corpus]
bm25 = fastbm25(corpus)

In [28]:
%%time
# query='cognitive science and artificial intelligence and evolutionary biology'
# weights=[0,0,0,2,2,0,5,5]
# tokenized_query = query.lower().strip().split(" ")
tokenized_query=query.translate(str.maketrans('', '', string.punctuation)).replace('\n',"").lower().split()
result = bm25.top_k_sentence_weighted(tokenized_query,weights,k=10)
sorted_scores=[r[1] for r in result]
top_n = [dataset['text'][i] for i in sorted_scores[1:10+1]]
    
result

CPU times: user 919 ms, sys: 2.57 ms, total: 922 ms
Wall time: 924 ms


[(['southern',
   'district',
   'of',
   'west',
   'virginia',
   'vacated',
   'by',
   'the',
   'retirement',
   'of',
   'robert',
   'jackson',
   'staker',
   'goodwin',
   'was',
   'confirmed',
   'by',
   'the',
   'united',
   'states',
   'senate',
   'on',
   'may',
   '8',
   '1995',
   'and',
   'received',
   'his',
   'commission',
   'on',
   'may',
   '10',
   '1995',
   'he',
   'served',
   'as',
   'chief',
   'judge',
   'of',
   'the',
   'district',
   'from',
   '2007',
   'to',
   '2012in',
   'june',
   '2017',
   'goodwin',
   'rejected',
   'prosecutors',
   'proposed',
   'plea',
   'bargain',
   'because',
   'he',
   'found',
   'holding',
   'an',
   'open',
   'jury',
   'trial',
   'of',
   'an',
   'accused',
   'fentanyl',
   'dealer',
   'would',
   'be',
   'in',
   'the',
   'public',
   'interestpersonalgoodwins',
   'son',
   'booth',
   'goodwin',
   'was',
   'appointed',
   'in',
   '2010',
   'by',
   'president',
   'barack',
   'obama',

CPU times: user 2.95 ms, sys: 1.43 ms, total: 4.38 ms
Wall time: 4.39 ms


[(['instituteadams',
   'lectures',
   'multiple',
   'biology',
   'classes',
   'at',
   'worcester',
   'polytechnic',
   'institute',
   'notably',
   'cell',
   'biology',
   'virology',
   'and',
   'advanced',
   'cell',
   'biology',
   'he',
   'is',
   'an',
   'avid',
   'supporter',
   'of',
   'abolishing',
   'textbooks',
   'for',
   'upper',
   'classes',
   'due',
   'to',
   'his',
   'belief',
   'that',
   'memorization',
   'does',
   'not',
   'contribute',
   'to',
   'a',
   'greater',
   'understanding',
   'of',
   'biologyresearch',
   'interestsmolecular',
   'medicineneurodegenerative',
   'diseasesneurotrophic',
   'factors',
   'as',
   'therapeutics',
   'for',
   'neuroregenerationmouse',
   'models',
   'for',
   'alzheimersreferencesexternal',
   'links',
   'wpis',
   'biology',
   'department',
   'website',
   'on',
   'david',
   'adamsyear',
   'of',
   'birth',
   'missing',
   'living',
   'peopleliving',
   'peopleworcester',
   'polytechnic',

## twbert adaptado

In [8]:
import re
import math

def score_vec(query, query_tf_vec, corpus, term_weights, avg_doc_len, k1=1.2, k3=8., b=0.75):
    # corpus = list of documents in word frequency format [{term: freq, ...}, {...}]
    query = re.findall(r"[a-z0-9']+", query)
    f_ti_t_w = term_weights * query_tf_vec
    num_docs = len(corpus)
    
    query_idf = {}
    for term in query:
        #print(corpus)
        df_t = sum([1 for doc_tf in corpus if term in doc_tf])
        query_idf[term] = math.log((num_docs - df_t + 0.5)/(df_t+0.5) + 1) # +1?
    
    doc_scores = list()
    for doc_tf in corpus:
        doc_len = sum(doc_tf.values())
        doc_freq=[]
        for term in query:
            if term in doc_tf:
                doc_freq.append(doc_tf[term])
            else:
                doc_freq.append(0)
        # doc_tf_vec = torch.Tensor([doc_tf[term] for term in query]) # mask?
        doc_tf_vec = torch.Tensor(doc_freq)
        num = doc_tf_vec * (k3 + 1) * f_ti_t_w
        k = k1 * ((1-b) + b * doc_len/avg_doc_len) + doc_tf_vec
        den = (k3 + f_ti_t_w) * k
        idf = torch.Tensor([query_idf[term] for term in query])
        doc_scores.append(torch.sum(idf * num/den))

    return torch.stack(doc_scores)

In [9]:
#count vectorizer
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer(max_features=1000)
count_vectorizer.fit(dataset['text'])

CountVectorizer(max_features=1000)

In [10]:
corpus=[{term:freq for term,freq in zip(count_vectorizer.get_feature_names_out(),doc_tf)} for doc_tf in count_vectorizer.transform(dataset['text']).toarray()]
avg_doc_len=84

In [13]:
def search(query,term_weights):
    query_tf_vec=[]
    for i in range(len(query.split())):
        if query.split()[i] in count_vectorizer.get_feature_names_out():
            query_tf_vec.append(count_vectorizer.transform([query])[0,i])
        else:
            query_tf_vec.append(0)
    query_tf_vec= torch.Tensor(query_tf_vec)
    scores=score_vec(query, query_tf_vec, corpus, term_weights, avg_doc_len)
    return scores.argsort(descending=True)[:RETRIEVAL_NUM]

query='a casa caiu'
term_weights= torch.Tensor([0 for i in range(len(query.split()))])
search(query,term_weights)

tensor([6670, 6661, 6662, 6663, 6664])

## twbert original

In [39]:
pip install polars

  Obtaining dependency information for polars from https://files.pythonhosted.org/packages/e3/60/1daa56ba2b2d49a84a71aea9b6a8241aa16b281d8f6d19d2f86fbbcc0ea7/polars-0.19.9-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.6/27.6 MB 4.1 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [40]:
import polars as pl
import numpy as np
import json

import torch
from torch.utils.data import Dataset, DataLoader


from pathlib import Path
from collections import Counter
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

from joblib import Parallel, delayed


In [43]:
stemmer = PorterStemmer()
sw = list(stopwords.words('english'))

In [44]:
def clean_and_count(pid, passage):
	s = [stemmer.stem(w) for w in passage if w not in sw]
	c = Counter(s)
	return (pid,c)

def clean_query(s):
	s = [stemmer.stem(w) for w in s if w not in sw]
	return s

def query_tf_vec(s):
	c = Counter(s)
	s = [c[w] for w in s]
	return s

In [45]:
class MSMARCOData(Dataset):
	def __init__(self, queries):
		self.queries = queries
		
	def __len__(self):
		return len(self.queries)
	
	def __getitem__(self, index):
		qid = self.queries[index]
		query = qid_map[qid]
		query_tf_vec = query_tf_vecs_map[qid]
		corpus = [doc_wf[d] for d in query_doc_map[qid]]
		targets = query_labels_map[qid]
		return query, query_tf_vec, corpus, targets
	


In [46]:
!wget https://msmarco.blob.core.windows.net/msmarcoranking/top1000.dev.tar.gz

--2023-10-18 22:59:29--  https://msmarco.blob.core.windows.net/msmarcoranking/top1000.dev.tar.gz
Resolvendo msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)... 20.150.34.4
Conectando-se a msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)|20.150.34.4|:443... conectado.
A requisição HTTP foi enviada, aguardando resposta... 200 OK
Tamanho: 687414398 (656M) [application/x-gzip]
Salvando em: “top1000.dev.tar.gz”

top1000.dev.tar.gz  100%[===================>] 655,57M  1,18MB/s    em 9m 19s  

2023-10-18 23:08:49 (1,17 MB/s) - “top1000.dev.tar.gz” salvo [687414398/687414398]



In [47]:
!wget https://msmarco.blob.core.windows.net/msmarcoranking/qrels.dev.tsv

--2023-10-18 23:08:49--  https://msmarco.blob.core.windows.net/msmarcoranking/qrels.dev.tsv
Resolvendo msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)... 20.150.34.4
Conectando-se a msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)|20.150.34.4|:443... conectado.
A requisição HTTP foi enviada, aguardando resposta... 200 OK
Tamanho: 1201626 (1,1M) [text/tab-separated-values]
Salvando em: “qrels.dev.tsv”

qrels.dev.tsv       100%[===================>]   1,15M   721KB/s    em 1,6s    

2023-10-18 23:08:52 (721 KB/s) - “qrels.dev.tsv” salvo [1201626/1201626]



In [49]:
torch.set_default_device('cuda')

qrels_dev = pl.read_csv("qrels.dev.tsv", separator='\t', has_header=False,new_columns=["qid", "i", "pid", "label"])
top1000 = pl.read_csv("top1000.dev", separator='\t', has_header=False, new_columns=["qid", "pid", "query", "passage"])

data = top1000.join(qrels_dev[['qid', 'pid','label']], on=['qid', 'pid'], how='left').fill_null(0).sort('label', descending=True).group_by('qid').head(100)
queries = data.select(pl.col("qid")).unique().to_numpy().squeeze()
train_queries = queries[:-500]
val_queries = queries[-500:]

query_tf_vecs = data.select(pl.col("qid"), pl.col('query')).unique().with_columns([
    pl.col("query").str.extract_all(r"[A-Za-z0-9']+").alias("query_terms")]).with_columns([
        pl.col("query_terms").map_elements(clean_query).alias("clean_query_terms")]).with_columns([
            pl.col("clean_query_terms").map_elements(query_tf_vec).alias("query_tf_vec"),
            pl.col("clean_query_terms").list.join(" ").alias("clean_query")])
        

In [50]:
doc_lists = data.select(pl.col("pid"), pl.col('passage')).unique().with_columns([
    pl.col("passage").str.to_lowercase().str.extract_all(r"[A-Za-z0-9']+").alias("passage_terms")]).select(pl.col("pid", "passage_terms"))

# parallelize the loop using joblib
doc_wf = dict(Parallel(n_jobs=-1)(delayed(clean_and_count)(doc[0], doc[1]) for doc in doc_lists.rows()))
        
query_doc_map = {row[0]: row[1] for row in data.select(pl.col("qid"), pl.col("pid")).group_by("qid").agg(pl.col("pid")).rows()}
query_labels_map = {row[0]: row[1] for row in data.select(pl.col("qid"), pl.col("label")).group_by("qid").agg(pl.col("label")).rows()}
qid_map = {row[0]: row[1] for row in query_tf_vecs.select(pl.col("qid"), pl.col("clean_query")).unique().rows()}
query_tf_vecs_map = {row[0]: row[1] for row in query_tf_vecs.select(pl.col("qid"), pl.col("query_tf_vec")).rows()}

train_dataset = MSMARCOData(train_queries)
val_dataset = MSMARCOData(val_queries)

In [60]:
def score_vec(query, query_tf_vec, corpus, term_weights, avg_doc_len, k1=1.2, k3=8., b=0.75):
    # corpus = list of documents in word frequency format [{term: freq, ...}, {...}]
    query = re.findall(r"[a-z0-9']+", query)
    f_ti_t_w = term_weights * query_tf_vec
    num_docs = len(corpus)
    
    query_idf = {}
    for term in query:
        #print(corpus)
        df_t = sum([1 for doc_tf in corpus if term in doc_tf])
        query_idf[term] = math.log((num_docs - df_t + 0.5)/(df_t+0.5) + 1) # +1?
    
    doc_scores = list()
    for doc_tf in corpus:
        doc_len = sum(doc_tf.values())
        doc_tf_vec = torch.Tensor([doc_tf[term] for term in query]) # mask?
        num = doc_tf_vec * (k3 + 1) * f_ti_t_w
        k = k1 * ((1-b) + b * doc_len/avg_doc_len) + doc_tf_vec
        den = (k3 + f_ti_t_w) * k
        idf = torch.Tensor([query_idf[term] for term in query])
        doc_scores.append(torch.sum(idf * num/den))
    
    return torch.stack(doc_scores)

In [74]:
accum_iter = 500
for i in range(10):
    for j in range(len(train_dataset)):
        query, query_tf_vec, corpus, target = train_dataset[j]
        query_tf_vec = torch.tensor(query_tf_vec, dtype=torch.float32, device="cpu")
        avg_doc_len = 500
        term_weights = torch.tensor([0 for i in range(len(query.split()))], dtype=torch.float32, device="cpu")
        scores=score_vec(query, query_tf_vec, corpus, term_weights, avg_doc_len)
        print(scores)
        a=2/0

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.])


ZeroDivisionError: division by zero

In [75]:
query_tf_vec.shape

torch.Size([6])

In [52]:
query

'texa requir need appli becom broker'

In [55]:
len(query_tf_vec)

6

In [56]:
corpus[0]

Counter({'requir': 3,
         'bachelor': 2,
         'degre': 2,
         'texa': 1,
         'state': 1,
         'secur': 1,
         'board': 1,
         'establish': 1,
         'educ': 1,
         'becom': 1,
         'stockbrok': 1,
         'mani': 1,
         'broker': 1,
         'dealer': 1,
         'firm': 1,
         'applic': 1,
         'sponsorship': 1,
         'hold': 1,
         'addit': 1,
         'choos': 1,
         'pursu': 1,
         'profession': 1,
         'design': 1,
         'career': 1,
         'minimum': 1,
         'usual': 1})

In [58]:
len(target)

100